#### _Ev Fiyatları_
- _Önceki dersteki modelinizi yeniden uygulayın._
- _Aynı model tanımını kullanarak OLS, Lasso, Ridge ve ElasticNet regresyonlarını deneyin. Bu kez, modelleriniz için en iyi hiperparametre değerlerini seçmek için k-fold çapraz doğrulama yapmanız gerekir. Hangi model en iyisidir? Açıklayın?_

> Önceki ödevin üzerinden devam edilmiştir.

In [10]:
Y = ev['SalePrice']
X = ev[['OverallQual', 'GrLivArea', 'GarageArea', 'TotalBsmtSF', '1stFlrSF', 
         'HeatingQC_TA', 'Foundation_CBlock', 'KitchenQual_TA', 'ExterQual_TA']]

X = [pd.concat([X**i for i in range(1,21)], axis=1)]

In [13]:
def regstats(col, Y, perc, regType):  #'ols', 'ridge', 'lasso', 'elasticnet'
    
    pf = pd.DataFrame(columns=['model', 'rsq_train', 'rsq_test', 'subt_rsq', 'mae_test', 'mse_test', 'rmse_test', 'mape_test']) 
    pd.options.display.float_format = '{:.3f}'.format
    
    
    for num,X in enumerate(col,1): 
        x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = perc, random_state = 42)
        
        if str(regType) == 'ols':
            results = LinearRegression().fit(x_train, y_train)
        elif str(regType) == 'ridge':
            results = Ridge(alpha=10**37).fit(x_train, y_train)
        elif str(regType) == 'lasso':
            results = Lasso(alpha=10**20.5).fit(x_train, y_train)
        elif str(regType) == 'elasticnet':
            results = ElasticNet(alpha=10**21, l1_ratio=0.5).fit(x_train, y_train)

        y_pred = results.predict(x_test)
        
        pf.loc[num] = (str(num)+'_'+regType,
                       results.score(x_train, y_train),
                       results.score(x_test, y_test),
                       results.score(x_train, y_train) - results.score(x_test, y_test),
                       mean_absolute_error(y_test, y_pred), 
                       mse(y_test, y_pred), 
                       rmse(y_test, y_pred), 
                       (np.mean(np.abs((y_test - y_pred) / y_test)) * 100))
    return pf

In [14]:
dfstats = pd.concat([regstats(X, Y, 0.2, 'ols'),
                     regstats(X, Y, 0.2, 'ridge'),
                     regstats(X, Y, 0.2, 'lasso'),
                     regstats(X, Y, 0.2, 'elasticnet')],
                     axis=0,ignore_index=True)
dfstats

,model,rsq_train,rsq_test,subt_rsq,mae_test,mse_test,rmse_test,mape_test
0,1_ols,0.849,0.787,0.062,23284.609,1152463270.546,33947.949,14.119
1,1_ridge,0.348,0.087,0.261,54005.108,4931923136.737,70227.652,35.919
2,1_lasso,0.611,0.506,0.105,39306.834,2670936473.057,51681.104,27.742
3,1_elasticnet,0.569,0.438,0.131,42500.494,3037809500.475,55116.327,29.576


- Modelin kapasitesi düşük olduğundan, açıklanırken zorlanıldı. Bu sebeple model kapasitesi X^21 ile artırıldı.
- Ridge modelinde r-sq değeri, aradaki farktan dolayı en ideal olanıdır.
- alpha değerleri azaltılırsa, daha uygun sonuçlar alınmaktadır. çapraz doğrulama?